## Import libraries

In [81]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Load data and prepare it

In [82]:
mnist_data , mnist_info = tfds.load(name = 'mnist' , with_info=True , as_supervised=True)
mnist_train , mnist_test = mnist_data['train'] , mnist_data['test']
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples , tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples , tf.int64)

In [83]:
# Scaling the data between(0,1)
def scale(image , label):
  image = tf.cast(image , tf.float32)
  image /=255.
  return image , label

scaled_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [84]:
# shuffle data
buffer_size = 10000
shuffled_data = scaled_data.shuffle(buffer_size)
validation_data = shuffled_data.take(num_validation_samples)
train_data = shuffled_data.skip(num_validation_samples)

batch_size = 100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs , validation_targets = next(iter(validation_data))

## Model

In [85]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(hidden_layer_size , activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size , activation='relu'),
        tf.keras.layers.Dense(output_size , activation = 'softmax')
])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [86]:
# Compile the model
model.compile(optimizer='adam' , loss = 'sparse_categorical_crossentropy' , metrics =['accuracy'])

In [89]:
# fit the model
num_epochs = 5

model.fit(train_data , epochs = num_epochs , validation_data = (validation_inputs , validation_targets) , verbose= 2)

Epoch 1/5
540/540 - 5s - 9ms/step - accuracy: 0.9821 - loss: 0.0574 - val_accuracy: 0.9825 - val_loss: 0.0563
Epoch 2/5
540/540 - 3s - 6ms/step - accuracy: 0.9841 - loss: 0.0523 - val_accuracy: 0.9823 - val_loss: 0.0567
Epoch 3/5
540/540 - 3s - 6ms/step - accuracy: 0.9857 - loss: 0.0473 - val_accuracy: 0.9833 - val_loss: 0.0533
Epoch 4/5
540/540 - 5s - 9ms/step - accuracy: 0.9872 - loss: 0.0425 - val_accuracy: 0.9858 - val_loss: 0.0467
Epoch 5/5
540/540 - 5s - 10ms/step - accuracy: 0.9886 - loss: 0.0382 - val_accuracy: 0.9843 - val_loss: 0.0491


## Test the model

In [90]:
test_loss , test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step - accuracy: 0.9719 - loss: 0.0926


In [91]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.19%
